# Demo notebook for Kamodo Flythrough "ModelFlythrough" function
The ModelFlythough function flies a user-supplied trajectory through the chosen model data. This is the function the other flythrough functions call once a trajectory is acquired.
You may run the notebook as is if you have the sample data file, but you must
change the 'file_dir', 'output_name', and 'plot_output' variables in block 3 to have the correct file path.

In [1]:
#import satellite flythrough code
from kamodo_ccmc.flythrough import SatelliteFlythrough as SF
import kamodo_ccmc.flythrough.model_wrapper as MW
#The testing data file is available at https://drive.google.com/file/d/1pHx9Q8v4vO59_RUMX-SJqYv_-dE3h-st/view?usp=sharing

C:\Users\rringuet\.conda\envs\Kamodo_env\lib\site-packages\spacepy\time.py:2365: UserWarning: Leapseconds may be out of date. Use spacepy.toolbox.update(leapsecs=True)
  warnings.warn('Leapseconds may be out of date.'


In [2]:
help(SF.ModelFlythrough)

Help on function ModelFlythrough in module kamodo_ccmc.flythrough.SatelliteFlythrough:

ModelFlythrough(model, file_dir, variable_list, sat_time, c1, c2, c3, coord_sys, high_res=20.0, output_name='', plot_coord='GEO', verbose=False, _print_units=True)
    Call satellite flythrough wrapper specific to the model chosen.
    Parameters:   
    ------------
    model: 'CTIPe','IRI', ...
    file_dir: complete path to where model data files are stored
    variable_list: List of standardized variable names. Corresponding integers 
        are allowed. See model variable output for details.
    sat_time: a numpy array of the utc timestamps
    c1, c2, c3: numpy arrays of the positions correlating to the utc timestamps
        (c1, c2, c3) should be (x,y,z) in R_E for cartesian coordinates, and (lon, lat, 
        radius (R_E) or altitude (km)) for spherical coordinates. 
    coord_sys: one of 'GDZ', 'GEO', 'GSM', 'GSE', 'SM', 'GEI', 'MAG', 'SPH', 'RLL'
        combined with '-sph' or '-car'. 

In [3]:
#Choosing input values for ModelFlythrough function call
#----------------------------  
model = 'TIEGCM'  #Choose which model to view the example for
file_dir = 'C:/Users/rringuet/Kamodo_Data/TIEGCM/Data/'  #full file path to where the model output data is stored
variable_list = ['T_n','rho','TEC','W_Joule','HmF2']  #list of desired variable names from above list, must be in list form 
#not all variables in the list will be available in the file(s) found.
coord_sys = 'GEO-sph'  #GEO spherical coordinates as the sample coordinate system for trajectory
#See https://sscweb.gsfc.nasa.gov/users_guide/Appendix_C.shtml for a description of coordinate types
#Choose from any option available in SpacePy.

#choose naming convention for output files
output_name = 'C:/Users/rringuet/Kamodo_env/ModelFlythroughExample_TIEGCM.nc' #filename for DATA output with extension
plot_coord = 'GSE'  #coordinate system chosen for output plots

In [4]:
#Option 1 for creating input trajectory coordinate arrays:
#create sample coordinate input arrays, geostationary-like orbit for simplicity (but much lower altitude)
import numpy as np
sat_time = np.linspace(1506039600,1506124800+86400*7,int(86400*7/30+1), dtype=int)  
#selected time range available in data @ 30 sec intervals
#The chosen time range should match the length of time in the model data files.
#See the times.csv file in the directory where the model data is stored for the available time ranges
#The file will appear after attempting to execute a flythrough function.
#Time values found not to be contained in the model data are automatically discarded (see output of block 10).

sat_lon = np.repeat(0., int(86400*7/30+1))  #in degrees
sat_lat = np.repeat(0., int(86400*7/30+1))  #in degrees
sat_radius = np.repeat(1.08, int(86400*7/30+1))  #in R_earth (~500 km altitude)
#if coord_grid is chosen to be cartesian ('car'), then lon, lat, radius -> x, y, z in the chosen coordinate system
#coordinate arrays must all be the same length
print(len(sat_time), len(sat_lon), len(sat_lat), len(sat_radius))

#convert sat_radius to km and print for sanity check
from astropy.constants import R_earth
sat_alt = sat_radius[0]*R_earth.value-R_earth.value  #in meters
print(sat_alt/1000., 'km')  #Rough altitude printed in km

20161 20161 20161 20161
510.248 km


In [5]:
#Option 2 for creating input trajectory coordinate arrays:
#pull a real satellite trajectory and just change the time values to match the model data.
help(SF.SatelliteTrajectory)

Help on function SatelliteTrajectory in module kamodo_ccmc.flythrough.SatelliteFlythrough:

SatelliteTrajectory(dataset, start_ts, stop_ts, coord_type='GEO', verbose=False)
    Retrieve and return satellite trajectory from HAPI/CDAWeb
    Parameters:
    ----------
    dataset: name of the satellite data set to pull trajectory from
    start_ts: utc timestamp for start of desired time interval
    stop_ts: utc timestamp for end of desired time interval    
    coord_type: Pick from GEO, GSM, GSE, or SM
    verbose: Set to true to be overwhelmed with information.
    
    Coordinates are retrieved on a cartesian grid.
    See kamodo_ccmc.flythrough.utils.ConvertCoord for info on the coordinate systems.



In [6]:
#Demonstrate how to use the flythrough function to alter a real satellite trajectory
#---------------------------------------------------------------------------
#The time series for each coordinate is returned in a dictionary
#Use https://sscweb.gsfc.nasa.gov/ to find the satellite name and time range desired
#The chosen time range should match the length of time in the model data files.
#See the times.csv file in the directory where the model data is stored for the available time ranges
#The file will appear after attempting to execute a flythrough function.
#Time values found not to be contained in the model data are automatically discarded (see output of next block).
traj_dict, new_coord_sys = SF.SatelliteTrajectory('grace1', 1426660000.0, 1426880700.0, coord_type='GSE')

#change the time range by subtracting the appropriate value (the diff between the first in each)
start1, start2 = min(sat_time), min(traj_dict['sat_time'])  #start1 can equal the minimum UTC timestamp from the csv file
if start1>start2: 
    diff = start1-start2
    new_time = traj_dict['sat_time']+diff
if start1<start2: 
    diff = start2-start1
    new_time = traj_dict['sat_time']-diff
#The cartesian spatial coordinate time arrays are x=traj_dict['c1'], y=traj_dic['c2'], and z=traj_dict['c3']

Attribute/Key names of return dictionary: dict_keys(['sat_time', 'c1', 'c2', 'c3'])


In [7]:
#run ModelFlythrough with user-created trajectory
#results = SF.ModelFlythrough(model, file_dir, variable_list, sat_time, sat_lon, 
#                            sat_lat, sat_radius, coord_sys,
#                           output_name=output_name, plot_coord=plot_coord)

#run ModelFlythrough with altered grace1 trajectory from SSCWeb
results = SF.ModelFlythrough(model, file_dir, variable_list, new_time, traj_dict['c1'], 
                            traj_dict['c2'], traj_dict['c3'], new_coord_sys,
                           output_name=output_name, plot_coord=plot_coord)
#open plots in separate internet browser window for interactivity. Nothing will open here.

Please remove all pxxx.nc files if present.
Please remove all pxxx.nc files if present.
2237 times are not in model output files and are excluded from the flythrough.


C:\Users\rringuet\.conda\envs\Kamodo_env\lib\site-packages\spacepy\coordinates.py:242: DeprecationWarning: No coordinate backend specified; using IRBEM. This default will change in the future.
  DeprecationWarning)
c:\users\rringuet\kamodo_env\kamodo\kamodo_ccmc\readers\tiegcm_4D.py:370: UserWarning: WARNING: missing_value not used since it
cannot be safely cast to variable data type
  for key in gvar_list}



Best height resolution achieved: 0.00575 m
Worst height resolution achieved: 19.59163 m


Best height resolution achieved: 0.02743 m
Worst height resolution achieved: 19.71656 m


Best height resolution achieved: 0.00116 m
Worst height resolution achieved: 19.68833 m


Best height resolution achieved: 0.00216 m
Worst height resolution achieved: 19.62668 m



c:\users\rringuet\kamodo_env\kamodo\kamodo_ccmc\flythrough\SF_utilities.py:296: RuntimeWarning: All-NaN slice encountered
  max_height, min_height = nanmax(rough_height), nanmin(rough_height)  #save for output, ignore NaNs if possible



Best height resolution achieved: 0.00173 m
Worst height resolution achieved: 19.21641 m


Best height resolution achieved: 0.00283 m
Worst height resolution achieved: 19.93211 m

{'T_n': 'K', 'rho': 'g/cm**3', 'TEC': '1/cm**2', 'HmF2': 'km', 'utc_time': 's', 'net_idx': '', 'c1': 'R_E', 'c2': 'R_E', 'c3': 'R_E'}
Please remove all pxxx.nc files if present.
Output saved in C:/Users/rringuet/Kamodo_env/ModelFlythroughExample_TIEGCM.nc.
Generating interactive plots...
-saving html div file:  C:/Users/rringuet/Kamodo_env/ModelFlythroughExample_TIEGCM_T_n_3D.html
-saving html div file:  C:/Users/rringuet/Kamodo_env/ModelFlythroughExample_TIEGCM_T_n_1D.html
-saving html div file:  C:/Users/rringuet/Kamodo_env/ModelFlythroughExample_TIEGCM_rho_3D.html
-saving html div file:  C:/Users/rringuet/Kamodo_env/ModelFlythroughExample_TIEGCM_rho_1D.html
-saving html div file:  C:/Users/rringuet/Kamodo_env/ModelFlythroughExample_TIEGCM_TEC_3D.html
-saving html div file:  C:/Users/rringuet/Kamodo_env/Mod